## Mapping and Geographic Investigation 

Capturing physical landscape traits in a written form is perhaps one of the oldest and most common examples
of information visualization still in use today. Geographical Information Systems (GIS) are high specialized
and complex, and made up of a myriad of unique techniques and tools. In this lecture I want to dip our toes
into this world, and show you how you can leverage geographical information to lead to insight in
computational narratives.

In [ ]:
# Let's start with bringing in a few of our common libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Now we'll setup matplotlib
%matplotlib inline

In [ ]:
# And we'll set some matplotlib defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [16.0,8.0]

In [ ]:
# The first bit of data I want to share with you is a single session of running data. Actually, this is a
# small piece of data from your project data source. Let's read in and take a look.
df=pd.read_csv("run.csv")
df.head()

In [ ]:
# Ok, so this data is made up of a latitude and longitude, a timestamp on what looks to be a second frequency
# as well as power, in watts, and heart rate, in beats per minute, and an enhanced_altitude probably in feet.
# I think that the first thing we need to consider when mapping is this longitude and lattitude. These look
# like really odd numbers, and it turns out that this data source is trying to capture the maximum prescision
# possible in an unsigned integer, so we need to convert these to more traditional decimal format. To be
# honest, I don't really know anything about this data format, but a little bit of googling suggests that this
# is easily done by multiplying by 180 degrees then dividing by 2 to the power of 31, so let's do that
df["position_lat_degrees"] = df["position_lat"] * ( 180 / 2**31 )
df["position_long_degrees"] = df["position_long"] * ( 180 / 2**31 )

# Now, it turns out we're not really done with this. You see, the earth is round(ish), but we are going to try
# and look at it on the screen which is flat(ish). My wife, who has a physical geography degree, gave me a
# mini lecture on all the great and awesome ways you can take global positions like longitude and latitude and
# convert them for display on flat things like maps. I'm going to save you from that (feel free to look up map
# projections on wikipediate) and tell you that we're going to use the Mercantor projection. Even better, I'm
# going to share with you the code to convert from latitude in degrees to a flat Mercantor projection which
# comes courtesy of the Open Street Map effort at https://wiki.openstreetmap.org/wiki/Mercator

import math
def lat2y(a):
  return 180.0/math.pi*math.log(math.tan(math.pi/4.0+a*(math.pi/180.0)/2.0))
df["position_lat_degrees_mercantor"]=df["position_lat_degrees"].apply(lat2y)

# And lastly, let's drop anything with missing values. This is a simplification for the moment, and might not
# be what you actually want to do in practice
df=df.dropna()
df.head()

In [ ]:
# The first approach I want to show you is probably the most simple. The gist is that we will render an image
# behind an Axes object, then just use our regular plotting on the Axes object. For this to work it means we
# need an image and we need to know the coordinates of the image bounds. Then we can set the "extent", which
# represents the bounds of the map. This means our image behind the Axes object will be using the same
# coordinate system as the Axes object itself, and the plot will be locked.

# First the image. I got mine through an export from Open Street Map and saved it in map.png. You can get a
# map directly from http://www.openstreetmap.org. To display this, we use the pyplot imread() function and
# pair it with imshow()
image=plt.imread("map.png")
plt.imshow(image, alpha=0.5, extent=[-83.77141,-83.75977,46.75230,46.76620])

In [ ]:
# Ok, great! That created a map, and we can see the X and Y axes are constrained by the extent that I setup.
# Now, the extent doesn't really matter, you can choose whatever you want, just make sure it aligns with
# whatever your map is. In this case, I looked for the min/max of my different fields in the data, then went
# to open street map and found a map that at least covered those bounds, because I want to show the whole
# data.

In [ ]:
# Now it's actually really easy to overlay our data on top of this plot, we just use whatever plotting
# function which exists in pyplot that we are interested in! In this case I'm going to use scatter(). I'm
# going to add a color bar as well, and change the values of the dots being plotted based on the power column
# in our data.

# Reshow the image, because we didn't turn off image closing in jupyter
plt.imshow(image, alpha=0.5, extent=[-83.77141,-83.75977,46.75230,46.76620])
# Plot our longitude and mercantor projected latitude data. We can set the series of data we want to be the
# colors of points using the c parameters, and we can choose from different color maps using the cmap
# parameter.
plt.scatter(df["position_long_degrees"],df["position_lat_degrees_mercantor"],
            s=5, c=df["Power"], cmap='Blues', alpha=0.75)
# Now we get pyplot to render a colorbar so we know the meeting of the colors
plt.colorbar().set_label("Power (watts)")
# And let's set a meaningful title
plt.suptitle("Power data from {} to {}".format(np.min(df["timestamp"]),np.max(df["timestamp"])),size='20');

In [ ]:
# Nice! We have a map, and we have some points plotted on the map showing the various power measurements! If
# you look at this you'll notice that many of the power colors look basically the same. What's going on here?
# Notice that the colorbar range goes from around 100 watts to 400 watts. It seems we have some outliers! Low
# power points which are skewing the automatic coloring. You probably have enough skills now to change this,
# why not pause the video and give it a try?

In [ ]:
# Ok, for the second approach, I want to see if we can bring in altitude as well, so we're going to look at
# a 3D plot. Let's import our 3D Axes library
from mpl_toolkits.mplot3d import Axes3D
# Now let's generate a 3D Axes to work with, we do this by calling the get current axes function and setting
# the projection to 3D
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax=plt.gca(projection='3d')

# Here I've captured the return value, which is the 3DAxes object. This acts a *lot* like the Axes object, but
# because it's a 3DAxes object pyplot convenience functions, like scatter(), aren't immediately available. But
# the process is basically the same as before, just that there is a third piece of data, the Z direction,
# which we'll set to the DataFrame enhanced_altitude column.
artists=ax.scatter(df["position_long_degrees"],df["position_lat_degrees_mercantor"],df["enhanced_altitude"],
                   s=5, c=df["Power"], cmap='Blues')

# You'll notice that I captured the return of the scatter() function as well, which is a list of Artists. This
# needs to be sent to colorbar so it knows how to calculate the range of values to show.
plt.colorbar(artists).set_label("Power (watts)")

# Now let's setup some axes labels
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Altitude')

In [ ]:
# Nice! We can see that the long tail is at a lower elevation than the movement at the top, which fits with
# personal information I have about the location (since I know this is up a hill!).

# One more example before we leave this data. It would be nice if we could see a range of different
# projections in this three dimensional space. One way to do that is with the matplotlib animation routines.
# First, let's setup our Axes3D again
fig = plt.figure()
ax = fig.gca(projection='3d')

# Now we define an initializeation function which will render the initial image, and return a set of Artists
# to render
def init():
    ax.scatter(df["position_long_degrees"],df["position_lat_degrees_mercantor"],df["enhanced_altitude"])
    return ax.get_children()

# The animation routines for matplotlib need to look through these children at each frame of the video. We
# need to define a function which also updates the view. In our case, all we are going to do is rotate the
# camera a bit. This is done with the Axes view_init() function, which takes two numbers determining rotation.
# Each time a frame is called for, our update function will be called with some content. In this case, we'll
# make this content just one of the parameters to view_init()
def update_view(frame):
    ax.view_init(30,frame)
    return ax.get_children()

# To start this whole process, we need the matplotlib animation routines
from matplotlib import animation

# And now we can setup the function which will call everything. Notice that I'll provide a list of frame
# numbers from 0 through 360 degrees in 3 degree increments. each item of this list will be passed through to
# my update_view() function after some time interval.
anim = animation.FuncAnimation(fig, update_view, init_func=init,
                               frames=[x for x in range(0,360,3)], interval=20)

# The last step is that we now need to start this and render it to the screen. The simpliest way to do this is
# with the Jupyter HTML() function, which displays some HTML. Then we can start anim running and ask it to
# generate an HTML 5-based video.
from IPython.display import HTML
HTML(anim.to_jshtml()) #also see to_html5_video() and save()

In [ ]:
# So that takes awhile to render, since matplotlib has to run through all of the frames and write them out
# to a javascript file to embed in the browser front end, managed by Jupyter. This part of the ecosystem is
# pretty raw actually, and could use some tweaking, but it's handy to also have a save() function for building
# animations you might want to look at afterwards. And why the first frame is rendered statically on the page
# as well as the video I'm not sure -- if you happen to find out feel free to let me know so I can update
# this lecture!

In [ ]:
# You know, it might be nice if that video could also look through a few different camera angles as well. We
# set it to 30 degrees, but maybe a second value could be passed in to frames to iterate upon. Why not pause
# the video and give it a try?

In [ ]:
# This method of creating a map using an image with extents is simple and reliable, though it's maybe not as
# fast and seemless as it could be. An alternative approach taken for maps in web systems is to use tile
# server. Tile servers actually create a matrix of maps at different zoom levels, then serve up portions of a
# map (the tiles) as requested from the client. This is how Google Maps, for instance, works, and it creates a
# responsive experience at the cost of being a bit more fragile, as network access is needed. This paradigm is
# also available in the Jupyter notebooks as well, through a project called leaflet. This project is all
# client side JavaScript which does the map requesting and rendering. To connect this to our python backend we
# can use the folium project. Let's walk through an example.

In [ ]:
# Let's import Folium
import folium
# Now let's render a spot from our previous data, for this we pick the center point of the map and a zoom
# level
m=folium.Map(location=[42.24,-83.764], zoom_start=12)
# A key eye will notice that I had to reverse our longitude and latitude for this library, *and* I'm not using
# the mercantor changed values for longitude. Welcome to geographical information systems!
display(m)

In [ ]:
# Immediately you'll notice that the user experience is nice, and you can see that the data is being streamed
# by the OpenStreetMap project (lower right corner).

# We can add callouts to the map using the Marker class, let's set this for our start and end.
m=folium.Map(location=[42.296,-83.768], zoom_start=15)
folium.Marker([df["position_lat_degrees"].iloc[0],df["position_long_degrees"].iloc[0]], 
              popup="Start").add_to(m)
folium.Marker([df["position_lat_degrees"].iloc[-1],df["position_long_degrees"].iloc[-1]], 
              popup="Stop").add_to(m)

# We also want to map the whole running route. The docs for folium point to a PolyLine as the appropriate
# class to use. The PolyLine takes a list of locations as tuples, which means we have to combine our latiude
# and longitude values pairwise, and this is easily achieved through the use of python's zip() function
route=folium.PolyLine(locations=zip(df["position_lat_degrees"],df["position_long_degrees"]),
                    weight=5,color='blue').add_to(m)

# Let's take a look at that
display(m)

In [ ]:
# Wow! What a nice rendering. We can zoom and look around and get a nice interactive sense of our data. And
# the docs really demonstrate how to use leaflet to do interesting things that have been well optimized in the
# browser and JavaScript world, including setting colors through HTML codes and events. And I'll be honest, I
# don't really know much about folium and how we might do something like, add interactivity through invents,
# or show popups to allow for exploration of individual data points, or even create a short video which might
# demo interesting aspects of the map. But these might be interesting places to take your final project, and
# I'm hoping a few of you do decide to show me something interesting with folium!

In [ ]:
# But before we leave mapping, I want to demonstrate one more information visualization technique that is
# common in data science and how folium supports it, and this is called the Choropleth. The Choropleth is
# basically a geographic heat map, where a map is rendered then a heat map of geopolitical boundaries is
# rendered on top with colors representing some value from the dataset.

In [ ]:
# Let's read in some sample data. I picked this data up from the Statistics Canada website on the provinces
# and territories and their populations.
df_pop=pd.DataFrame(
[["Newfoundland and Labrador",528817],
["Prince Edward Island",152021],
["Nova Scotia",953869],
["New Brunswick",759655],
["Quebec",8394034],
["Ontario",14193384],
["Manitoba",1338109],
["Saskatchewan",1163925],
["Alberta",4286134],
["British Columbia",4817160],
["Yukon",38459],
["Northwest Territories",44520],
["Nunavut",37996]], columns=["Province","Population"])

# And here's a dataset I got from the QuantHockey website. I thought maybe it would be interesting to go and
# look at which province tends to train the most NHL players per capita. I'm actually from a province right in
# the middle of Canada called Saskatchewan, and we don't have an NHL team which is a bit disappointing. We have
# to drive 6 hours to go watch a game live, either in Alberta (Edmonton or Calgary) or Manitoba (Winnipeg).
# Here's a link to the QuantHockey dataset: https://www.quanthockey.com/nhl/province-totals/nhl-players-career-stats.html
df_players=pd.read_csv("hockey_players_by_province.csv")

# Now let's merge these datasets together
df=df_players.merge(df_pop,on="Province")

# And calculate the ratio of NHL players by province population
df["Ratio"]=df["Players"]/df["Population"]
df.head()

In [ ]:
# So, making a Choropleth in folium is actually pretty easy, as long as you have a shape file for your
# geopolitical regions. Just like different map projections, shape files are a huge bag of hurt with multiple
# competing specifications and details on how to convert from one to another. To be honest, this is a massive
# time sink, but an emerging open standard which seems to be available now is called geojson. As the name
# implies, geojson files are all json coded polygons with embedded metadata. I've save the geopolitical borders
# for Canadian provinces in the cdn.json file.

# Ok, let's start by making a new map zoomed pretty far out - Canada's pretty big
m = folium.Map(location=[58,-92], zoom_start=3)

# Now we create the Choropleth object. The first parameter is geodata, which points to our json file. Then
# we indicate the pandas dataframe we want to use as the data, and we provide the two columns, where the first
# is a reference to the region we want to color and the second is the value we want to use for the heat map.
# We have to specify a color, then an important parameter called "key_on". This last parameter maps to the
# features in our json data file.
folium.Choropleth(geo_data="cdn.json", data=df, columns=["Province","Ratio"],
                  fill_color='YlOrRd',key_on='feature.properties.name').add_to(m)

# And now we just render our map.
display(m)

In [ ]:
# Nice, a Choropleth map. We can see that there are two regions which are gray, indicating that they have no
# data. We also see that folium automatically adds the colorbar at the top of the map. There's a lot of light
# yellow, indicating a small amount of NHL training, and then one province in particular seems to be quite
# high when producing NHL players. It turns out that maybe not having a home team means folks from
# Saskatchewan get out and play more hockey instead of just watching it from the couch.

In this lecture I've just touched on some of the information mapping you can do mixing geographical
structures and data. We've seen several different approaches, from rendering a simple data plot superimposed
over a map, to polygon coloring in the choropleth. Geographical Information Systems (GIS) are a big area,
but it's helpful to know a few things about spatial representation of data for those times when the physical
is a part of your investigation. 